## المقدمة

ستتناول هذه الدرس:
- ما هو استدعاء الدوال وحالات استخدامه
- كيفية إنشاء استدعاء دالة باستخدام Azure OpenAI
- كيفية دمج استدعاء دالة في تطبيق

## أهداف التعلم

بعد الانتهاء من هذا الدرس ستعرف وتفهم:

- الهدف من استخدام استدعاء الدوال
- إعداد استدعاء دالة باستخدام خدمة Azure OpenAI
- تصميم استدعاءات دوال فعّالة لحالة استخدام تطبيقك


## فهم استدعاء الدوال

في هذا الدرس، نرغب في بناء ميزة لشركتنا الناشئة في مجال التعليم تتيح للمستخدمين استخدام روبوت دردشة للعثور على الدورات التقنية. سنوصي بالدورات التي تناسب مستوى مهاراتهم، ودورهم الحالي، والتقنية التي يهتمون بها.

لإكمال ذلك، سنستخدم مزيجاً من:
 - `Azure Open AI` لإنشاء تجربة دردشة للمستخدم
 - `Microsoft Learn Catalog API` لمساعدة المستخدمين في العثور على الدورات بناءً على طلبهم
 - `Function Calling` لأخذ استفسار المستخدم وإرساله إلى دالة لإجراء طلب API.

للبدء، دعونا نلقي نظرة على سبب رغبتنا في استخدام استدعاء الدوال من الأساس:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # الحصول على رد جديد من GPT حيث يمكنه رؤية استجابة الدالة


print(second_response.choices[0].message)


### لماذا استخدام استدعاء الدوال

إذا كنت قد أنهيت أي درس آخر في هذه الدورة، فمن المحتمل أنك تدرك قوة استخدام نماذج اللغة الكبيرة (LLMs). ونأمل أيضًا أنك لاحظت بعض القيود لديها.

استدعاء الدوال هو ميزة في خدمة Azure Open AI تهدف إلى التغلب على القيود التالية:
1) تنسيق استجابة ثابت
2) القدرة على استخدام بيانات من مصادر أخرى في التطبيق ضمن سياق المحادثة

قبل وجود استدعاء الدوال، كانت الردود من نماذج اللغة الكبيرة غير منظمة وغير متسقة. كان على المطورين كتابة شيفرة تحقق معقدة للتأكد من قدرتهم على التعامل مع كل اختلاف في الردود.

لم يكن بإمكان المستخدمين الحصول على إجابات مثل "ما هو الطقس الحالي في ستوكهولم؟". وذلك لأن النماذج كانت محدودة بالبيانات التي تم تدريبها عليها فقط.

لنلقِ نظرة على المثال أدناه الذي يوضح هذه المشكلة:

لنفترض أننا نريد إنشاء قاعدة بيانات لمعلومات الطلاب حتى نتمكن من اقتراح الدورة المناسبة لهم. في الأسفل لدينا وصفان لطالبين يحتويان على بيانات متشابهة جدًا.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

نريد إرسال هذا إلى نموذج لغوي كبير لتحليل البيانات. يمكن استخدام ذلك لاحقًا في تطبيقنا لإرساله إلى واجهة برمجة التطبيقات أو تخزينه في قاعدة بيانات.

لنقم بإنشاء مطالبتين متطابقتين نوجه فيهما النموذج اللغوي حول المعلومات التي نهتم بها:


نريد إرسال هذا إلى نموذج لغوي كبير لتحليل الأجزاء المهمة لمنتجنا. حتى نتمكن من إنشاء مطالبتين متطابقتين لإعطاء التعليمات للنموذج اللغوي الكبير:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


بعد إنشاء هذين الطلبين، سنرسلهم إلى نموذج اللغة الكبير باستخدام `openai.ChatCompletion`. نقوم بتخزين الطلب في متغير `messages` ونعين الدور إلى `user`. هذا لمحاكاة رسالة من مستخدم يتم كتابتها إلى روبوت الدردشة.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

الآن يمكننا إرسال كلا الطلبين إلى نموذج اللغة الكبير وفحص الاستجابة التي نتلقاها.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

على الرغم من أن التعليمات متشابهة والوصف متقارب، إلا أنه يمكننا الحصول على صيغ مختلفة لخاصية `Grades`.

إذا قمت بتشغيل الخلية أعلاه عدة مرات، قد يكون الشكل إما `3.7` أو `3.7 GPA`.

يحدث هذا لأن نموذج اللغة الكبير يتعامل مع بيانات غير منظمة على شكل تعليمات مكتوبة ويعيد أيضاً بيانات غير منظمة. نحن بحاجة إلى تنسيق منظم حتى نعرف ما الذي نتوقعه عند تخزين أو استخدام هذه البيانات.

من خلال استخدام استدعاء الدوال، يمكننا التأكد من أننا نستلم بيانات منظمة. عند استخدام استدعاء الدوال، النموذج لا يقوم فعلياً باستدعاء أو تشغيل أي دوال. بدلاً من ذلك، نقوم بإنشاء هيكل للنموذج ليتبعه في ردوده. بعد ذلك نستخدم هذه الردود المنظمة لمعرفة أي دالة يجب تشغيلها في تطبيقاتنا.


![مخطط تدفق استدعاء الدالة](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ar.png)


### حالات استخدام استدعاء الدوال

**استدعاء الأدوات الخارجية**  
تُعتبر الدردشات الذكية ممتازة في تقديم الإجابات على أسئلة المستخدمين. من خلال استخدام استدعاء الدوال، يمكن للدردشة الذكية استخدام رسائل المستخدمين لتنفيذ مهام معينة. على سبيل المثال، يمكن لطالب أن يطلب من الدردشة الذكية "إرسال بريد إلكتروني إلى معلمي وأخبره أنني بحاجة إلى مزيد من المساعدة في هذا الموضوع". في هذه الحالة، يمكن استدعاء دالة مثل `send_email(to: string, body: string)`

**إنشاء استعلامات API أو قواعد بيانات**  
يمكن للمستخدمين البحث عن معلومات باستخدام اللغة الطبيعية التي يتم تحويلها إلى استعلام منسق أو طلب API. مثال على ذلك، معلم يطلب "من هم الطلاب الذين أكملوا الواجب الأخير"، ويمكن هنا استدعاء دالة باسم `get_completed(student_name: string, assignment: int, current_status: string)`

**إنشاء بيانات منظمة**  
يمكن للمستخدمين أخذ نص أو ملف CSV واستخدام نموذج اللغة الكبير لاستخلاص المعلومات المهمة منه. على سبيل المثال، يمكن لطالب تحويل مقالة من ويكيبيديا عن اتفاقيات السلام إلى بطاقات مراجعة ذكية بالذكاء الاصطناعي. يمكن تنفيذ ذلك باستخدام دالة مثل `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## ٢. إنشاء أول استدعاء دالة لك

عملية إنشاء استدعاء دالة تتضمن ثلاث خطوات رئيسية:
١. استدعاء واجهة برمجة التطبيقات لإكمال المحادثة مع قائمة الدوال الخاصة بك ورسالة المستخدم
٢. قراءة رد النموذج لتنفيذ إجراء مثل تنفيذ دالة أو استدعاء واجهة برمجة تطبيقات
٣. إجراء استدعاء آخر لواجهة برمجة التطبيقات لإكمال المحادثة مع الرد القادم من دالتك لاستخدام تلك المعلومات في إنشاء رد للمستخدم.


![تدفق استدعاء الدالة](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ar.png)


### عناصر استدعاء الدالة

#### إدخال المستخدم

الخطوة الأولى هي إنشاء رسالة من المستخدم. يمكن تعيين هذه الرسالة ديناميكياً من خلال أخذ قيمة من حقل إدخال نصي أو يمكنك تعيين قيمة هنا مباشرة. إذا كانت هذه هي المرة الأولى التي تتعامل فيها مع واجهة برمجة تطبيقات إكمال المحادثة، نحتاج إلى تحديد `role` و`content` للرسالة.

يمكن أن تكون قيمة `role` إما `system` (لإنشاء القواعد)، أو `assistant` (النموذج)، أو `user` (المستخدم النهائي). في حالة استدعاء الدوال، سنحددها كـ `user` مع مثال على سؤال.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### إنشاء الدوال.

الآن سنقوم بتعريف دالة ومعاملات هذه الدالة. سنستخدم هنا دالة واحدة فقط باسم `search_courses`، لكن يمكنك إنشاء عدة دوال.

**مهم** : يتم تضمين الدوال في رسالة النظام إلى نموذج اللغة الكبير، وسيتم احتسابها ضمن عدد الرموز المتاحة لديك.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**تعريفات**

`name` - اسم الدالة التي نريد استدعاءها.

`description` - هذا هو الوصف لكيفية عمل الدالة. من المهم هنا أن يكون الوصف محددًا وواضحًا.

`parameters` - قائمة بالقيم والصيغة التي تريد من النموذج أن ينتجها في رده.

`type` - نوع البيانات التي سيتم تخزين الخصائص فيها.

`properties` - قائمة بالقيم المحددة التي سيستخدمها النموذج في رده.

`name` - اسم الخاصية التي سيستخدمها النموذج في رده المنسق.

`type` - نوع البيانات لهذه الخاصية.

`description` - وصف الخاصية المحددة.

**اختياري**

`required` - خاصية مطلوبة لإتمام استدعاء الدالة.


### إجراء استدعاء الدالة
بعد تعريف الدالة، نحتاج الآن إلى تضمينها في الاستدعاء إلى واجهة برمجة تطبيقات إكمال الدردشة. نقوم بذلك عن طريق إضافة `functions` إلى الطلب. في هذه الحالة نكتب `functions=functions`.

هناك أيضًا خيار لتعيين `function_call` إلى `auto`. هذا يعني أننا سنسمح لنموذج اللغة الكبير بتحديد أي دالة يجب استدعاؤها بناءً على رسالة المستخدم بدلاً من تعيينها بأنفسنا.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

الآن دعونا نلقي نظرة على الاستجابة ونرى كيف تم تنسيقها:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

يمكنك أن ترى أن اسم الدالة تم استدعاؤه ومن خلال رسالة المستخدم، تمكن نموذج الذكاء الاصطناعي من إيجاد البيانات المناسبة لتعبئة معطيات الدالة.


## 3. دمج استدعاءات الدوال في التطبيق

بعد أن قمنا باختبار الاستجابة المنسقة من LLM، يمكننا الآن دمجها في التطبيق.

### إدارة سير العمل

لدمج هذا في تطبيقنا، دعونا نتبع الخطوات التالية:

أولاً، سنقوم باستدعاء خدمات Open AI وتخزين الرسالة في متغير باسم `response_message`.


In [ ]:
response_message = response.choices[0].message

الآن سنعرّف الدالة التي ستستدعي واجهة برمجة تطبيقات Microsoft Learn للحصول على قائمة الدورات:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



كأفضل ممارسة، سنرى بعد ذلك ما إذا كان النموذج يرغب في استدعاء دالة. بعد ذلك، سنقوم بإنشاء واحدة من الدوال المتاحة ونطابقها مع الدالة التي تم استدعاؤها.
بعدها سنأخذ معاملات الدالة ونربطها بالمعاملات القادمة من نموذج اللغة الكبير.

وأخيرًا، سنضيف رسالة استدعاء الدالة والقيم التي تم إرجاعها بواسطة رسالة `search_courses`. هذا يمنح نموذج اللغة الكبير كل المعلومات التي يحتاجها
للرد على المستخدم باستخدام لغة طبيعية.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## تحدي البرمجة

عمل رائع! لمواصلة تعلمك حول استدعاء الدوال في Azure Open AI يمكنك بناء: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - إضافة المزيد من معلمات الدالة التي قد تساعد المتعلمين في العثور على المزيد من الدورات. يمكنك العثور على معلمات الـ API المتاحة هنا:
 - إنشاء استدعاء دالة آخر يأخذ معلومات إضافية من المتعلم مثل لغته الأم
 - إنشاء معالجة للأخطاء عندما لا يُرجع استدعاء الدالة و/أو استدعاء الـ API أي دورات مناسبة



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
